In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import plotly.express as px
from scipy.stats import ttest_ind
from sklearn.decomposition import PCA
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as hc
from scipy.spatial import distance
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import dash_bio
import statsmodels.api as sm

# set infinite display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# allow tqdm to chart pandas progress
tqdm.pandas()

## This notebook contains the pipeline and all of its constituent tools.

### The dataframes need some further processing before the pipeline can run on them.

In [2]:
# import the TPM dataframe for the majority of the pipeline
TPM = pd.read_csv('../../results/TPM.tsv', sep='\t', index_col=0)

### Processing the TPM dataframe for the statistical tools

In [3]:
# transpose the dataframe
TPM = TPM.T
# rename TPM index to read_group_id
TPM.index.names = ['read_group_id']
# turn the index into a column
TPM.reset_index(inplace=True)
# preview the dataframe only showing the first 10 columns
TPM.iloc[:, :10].head()

Name,read_group_id,ENSMUST00000196221.2,ENSMUST00000179664.2,ENSMUST00000177564.2,ENSMUST00000178537.2,ENSMUST00000178862.2,ENSMUST00000179520.2,ENSMUST00000179883.2,ENSMUST00000195858.2,ENSMUST00000179932.2
0,SRR16522870_GSM5639190_RS24_RNA-seq_Mus_muscul...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SRR16522866_GSM5639186_RS11_RNA-seq_Mus_muscul...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SRR16522874_GSM5639194_RS5_RNA-seq_Mus_musculu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SRR16522873_GSM5639193_RS4_T_RNA-seq_Mus_muscu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SRR16522868_GSM5639188_RS18_RNA-seq_Mus_muscul...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# sum of TPM values for each transcript
TPM_sum = TPM.iloc[:, 1:].sum(axis=0)

In [5]:
# remove all rows in TPM_sum that have values below the 95th percentile
TPM_sum_threshold = TPM_sum[TPM_sum > TPM_sum.quantile(0.05)]

In [6]:
# compare shapes of the two dataframes
print(TPM_sum.shape)
print(TPM_sum_threshold.shape)

(115299,)
(99469,)


In [7]:
# Create a list of columns to keep
columns_to_keep = ['read_group_id'] + [col for col in TPM.columns[1:] if col in TPM_sum_threshold.index]

# Create a copy of TPM dataframe with only the desired columns
TPM_copy = TPM[columns_to_keep].copy()

In [8]:
# compare shapes of the two dataframes
print(TPM.shape)
print(TPM_copy.shape)

(24, 115300)
(24, 99470)


In [9]:
# make a TPM backup
TPM_backup = TPM.copy()
# replace TPM with TPM_copy
TPM = TPM_copy

In [10]:
#load metadata dataframe
metadata = pd.read_csv('../../results/metadata.tsv', sep='\t')
#preview the dataframe only showing the first 10 columns
metadata

,Run,source_name,cell_type,condition
0,SRR16522866,RS,Malignant B cells,RS
1,SRR16522867,RS,Malignant B cells,RS
2,SRR16522868,RS,Malignant B cells,RS
3,SRR16522869,RS,Malignant B cells,RS
4,SRR16522870,RS,Malignant B cells,RS
5,SRR16522871,RS,Malignant B cells,RS
6,SRR16522872,RS,Malignant B cells,RS
7,SRR16522873,RS,Malignant B cells,RS
8,SRR16522874,RS,Malignant B cells,RS
9,SRR16522875,RS,Malignant B cells,RS


In [11]:
# replace CLL_RS2, CLL_RS4, CLL_RS6, CLL_RS7 with CLL_RS
metadata['source_name'] = metadata['source_name'].replace(['CLL_RS2', 'CLL_RS4', 'CLL_RS6', 'CLL_RS7'], 'CLL_RS')

In [12]:
metadata

,Run,source_name,cell_type,condition
0,SRR16522866,RS,Malignant B cells,RS
1,SRR16522867,RS,Malignant B cells,RS
2,SRR16522868,RS,Malignant B cells,RS
3,SRR16522869,RS,Malignant B cells,RS
4,SRR16522870,RS,Malignant B cells,RS
5,SRR16522871,RS,Malignant B cells,RS
6,SRR16522872,RS,Malignant B cells,RS
7,SRR16522873,RS,Malignant B cells,RS
8,SRR16522874,RS,Malignant B cells,RS
9,SRR16522875,RS,Malignant B cells,RS


In [13]:
# remove the . and everything after it from the transcript_id columns in the TPM dataframe
TPM.columns = [col.split('.')[0] for col in TPM.columns]
#preview first 10 columns of TPM dataframe
TPM.iloc[:, :10].head()

,read_group_id,ENSMUST00000103569,ENSMUST00000103570,ENSMUST00000103571,ENSMUST00000198019,ENSMUST00000179789,ENSMUST00000178768,ENSMUST00000103580,ENSMUST00000103646,ENSMUST00000197754
0,SRR16522870_GSM5639190_RS24_RNA-seq_Mus_muscul...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,SRR16522866_GSM5639186_RS11_RNA-seq_Mus_muscul...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,SRR16522874_GSM5639194_RS5_RNA-seq_Mus_musculu...,0.0,0.994226,0.204338,0.0,0.0,0.0,0.0,0.0,0.0
3,SRR16522873_GSM5639193_RS4_T_RNA-seq_Mus_muscu...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,SRR16522868_GSM5639188_RS18_RNA-seq_Mus_muscul...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# do the same for the backup dataframe
TPM_backup.columns = [col.split('.')[0] for col in TPM_backup.columns]

In [15]:
# for TPM and TPM_backup split read_group_id on _ and keep the first one
TPM['read_group_id'] = TPM['read_group_id'].str.split('_').str[0]

In [16]:
TPM_backup['read_group_id'] = TPM_backup['read_group_id'].str.split('_').str[0]

In [17]:
TPM.iloc[:, :10].head()

,read_group_id,ENSMUST00000103569,ENSMUST00000103570,ENSMUST00000103571,ENSMUST00000198019,ENSMUST00000179789,ENSMUST00000178768,ENSMUST00000103580,ENSMUST00000103646,ENSMUST00000197754
0,SRR16522870,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,SRR16522866,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,SRR16522874,0.0,0.994226,0.204338,0.0,0.0,0.0,0.0,0.0,0.0
3,SRR16522873,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,SRR16522868,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


### Selecting a subset of relevant metadata columns to make the query process easier.

In [18]:
selected_metadata = metadata

### Loading the gene_name dictionaries.

In [19]:
with open('../../results/gene_to_transcript_mapping.pkl', 'rb') as file:
    gene_to_transcript_mapping = pickle.load(file)
with open('../../results/transcript_to_gene_mapping.pkl', 'rb') as file:
    transcript_to_gene_mapping = pickle.load(file)

### Setting up Plotly

In [57]:
import chart_studio
import chart_studio.plotly as py

# Set your credentials
chart_studio.tools.set_credentials_file(username='waqaas108', api_key='ZkYpY2kalOAcwPfZbo80')

## Pipeline

In [20]:
def modded_query_maker(dataframe, TPM_dataframe):
    print(TPM_dataframe.columns[:10])
    # set first column as index for TPM_dataframe
    TPM_dataframe.set_index(TPM_dataframe.columns[0], inplace=True)
    dataframe1 = dataframe.loc[(dataframe['source_name'] == 'RS')]
    dataframe2 = dataframe.loc[(dataframe['source_name'] == 'CLL_RS')]
    dataframe3 = dataframe.loc[(dataframe['source_name'] == 'CLL')]
    dataframe4 = dataframe.loc[(dataframe['source_name'] == 'B cell')]
    # use the case_id column to subset the TPM dataframe
    TPM_dataframe1 = TPM_dataframe[TPM_dataframe.index.isin(dataframe1['Run'])]
    TPM_dataframe2 = TPM_dataframe[TPM_dataframe.index.isin(dataframe2['Run'])]
    TPM_dataframe3 = TPM_dataframe[TPM_dataframe.index.isin(dataframe3['Run'])]
    TPM_dataframe4 = TPM_dataframe[TPM_dataframe.index.isin(dataframe4['Run'])]
    # print the shape of the two dataframes
    print('Dataframe 1 shape: ', TPM_dataframe1.shape)
    print('Dataframe 2 shape: ', TPM_dataframe2.shape)
    print('Dataframe 3 shape: ', TPM_dataframe3.shape)
    print('Dataframe 4 shape: ', TPM_dataframe4.shape)

    # return the TPM dataframes
    return TPM_dataframe1, TPM_dataframe2, TPM_dataframe3, TPM_dataframe4

In [67]:
from scipy.stats import f_oneway
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests
from tqdm import tqdm

def comprehensive_gene_analysis(gene_dict, df1, df2, df3, df4, genemap, log_threshold, alpha=0.1, post_online=False):
    """
    This function performs the full analysis pipeline: ANOVA at the transcript level,
    aggregation to the gene level, calculation of log fold changes and Euclidean distances,
    and visualization of significant genes. Optionally, it can post the figure online to Plotly Chart Studio.

    Parameters:
    - gene_dict: Dictionary mapping transcript IDs to gene names.
    - df1, df2, df3, df4: DataFrames containing TPM counts for each cohort.
    - genemap: Dictionary mapping gene names to lists of transcript IDs.
    - log_threshold: Log fold change threshold for significance.
    - alpha: Significance level for FDR correction.
    - post_online: Boolean flag to post the figure online through Plotly Chart Studio.

    Returns:
    - mean_df_min: DataFrame containing aggregated and visualized gene-level results.
    """

    # Step 1: Perform ANOVA at the transcript level
    df1 = df1.iloc[:, 1:]
    df2 = df2.iloc[:, 1:]
    df3 = df3.iloc[:, 1:]
    df4 = df4.iloc[:, 1:]

    anova_results = {}
    for col in tqdm(df1.columns, desc="Performing ANOVA", unit="transcript"):
        anova_results[col] = f_oneway(df1[col], df2[col], df3[col], df4[col])

    # Create DataFrame for ANOVA results
    anova_df = pd.DataFrame({
        'transcript_id': list(anova_results.keys()),
        'f_statistic': [result.statistic for result in anova_results.values()],
        'p_value': [result.pvalue for result in anova_results.values()],
    })

    # Apply FDR correction
    anova_df['adjusted_p_value'] = multipletests(anova_df['p_value'], method='fdr_bh')[1]

    # Identify significant transcripts after FDR correction
    anova_df['significant'] = anova_df['adjusted_p_value'] < alpha

    # Add gene names
    anova_df['gene_name'] = anova_df['transcript_id'].map(gene_dict)

    # Step 2: Aggregate transcript-level results to the gene level
    idx = anova_df.groupby('gene_name')['adjusted_p_value'].idxmin()
    gene_level_results_min = anova_df.loc[idx, ['gene_name', 'p_value', 'adjusted_p_value', 'significant']]

    # Step 3: Calculate log fold changes and Euclidean distances for visualization
    results = []
    for gene, transcripts in tqdm(genemap.items(), desc="Calculating Log FC and Euclidean Distances", unit="gene"):
        # Extract the relevant columns from the TPM dataframe
        gene_data1 = df1[[transcript for transcript in transcripts if transcript in df1.columns]]
        gene_data2 = df2[[transcript for transcript in transcripts if transcript in df2.columns]]
        gene_data3 = df3[[transcript for transcript in transcripts if transcript in df3.columns]]
        gene_data4 = df4[[transcript for transcript in transcripts if transcript in df4.columns]]

        # Compute sums for each cohort
        sum1 = gene_data1.sum(axis=1)
        sum2 = gene_data2.sum(axis=1)
        sum3 = gene_data3.sum(axis=1)
        sum4 = gene_data4.sum(axis=1)

        # Compute vectors for each cohort
        vector1 = gene_data1.sum(axis=0)
        vector2 = gene_data2.sum(axis=0)
        vector3 = gene_data3.sum(axis=0)
        vector4 = gene_data4.sum(axis=0)

        # Compute means for each cohort
        mean1 = sum1.mean()
        mean2 = sum2.mean()
        mean3 = sum3.mean()
        mean4 = sum4.mean()

        # Normalizing vectors using Euclidean distance
        n_sum1 = np.linalg.norm(vector1)
        n_sum2 = np.linalg.norm(vector2)
        n_sum3 = np.linalg.norm(vector3)
        n_sum4 = np.linalg.norm(vector4)

        # Get normalized vectors
        v_mean1 = vector1 / n_sum1
        v_mean2 = vector2 / n_sum2
        v_mean3 = vector3 / n_sum3
        v_mean4 = vector4 / n_sum4

        # Compute Euclidean distances for all pairwise comparisons
        euclidean_distances = {
            'B cell_vs_CLL': np.linalg.norm(v_mean1 - v_mean2),
            'B cell_vs_CLL_RS': np.linalg.norm(v_mean1 - v_mean3),
            'B cell_vs_RS': np.linalg.norm(v_mean1 - v_mean4),
            'CLL_vs_CLL_RS': np.linalg.norm(v_mean2 - v_mean3),
            'CLL_vs_RS': np.linalg.norm(v_mean2 - v_mean4),
            'CLL_RS_vs_RS': np.linalg.norm(v_mean3 - v_mean4)
        }

        # Compute log fold changes for all pairwise comparisons
        log2_fold_changes = {
            'B cell_vs_CLL': np.log2(mean1 / mean2) if mean1 > 0 and mean2 > 0 else np.nan,
            'B cell_vs_CLL_RS': np.log2(mean1 / mean3) if mean1 > 0 and mean3 > 0 else np.nan,
            'B cell_vs_RS': np.log2(mean1 / mean4) if mean1 > 0 and mean4 > 0 else np.nan,
            'CLL_vs_CLL_RS': np.log2(mean2 / mean3) if mean2 > 0 and mean3 > 0 else np.nan,
            'CLL_vs_RS': np.log2(mean2 / mean4) if mean2 > 0 and mean4 > 0 else np.nan,
            'CLL_RS_vs_RS': np.log2(mean3 / mean4) if mean3 > 0 and mean4 > 0 else np.nan
        }

        # Find the comparison with the highest Euclidean distance for the gene
        max_comp = max(euclidean_distances, key=euclidean_distances.get)

        # Append the result for the gene using the max comparison
        results.append({
            'gene_name': gene,
            'comparison': max_comp,
            'log2_fold_change': log2_fold_changes[max_comp],
            'euclidean_distance': euclidean_distances[max_comp]
        })

    # Convert results to a DataFrame
    mean_df = pd.DataFrame(results)

    # Merge with gene_level_results_min to include significance information
    mean_df_min = pd.merge(mean_df, gene_level_results_min[['gene_name', 'significant']], on='gene_name', how='left')

    # Calculate the 95th percentile for Euclidean distance
    percentile_95 = mean_df_min['euclidean_distance'].quantile(0.95)

    # Filter genes that are significant at the transcript level, at or above the 95th percentile, and within log thresholds
    filtered_genes = mean_df_min[
        (mean_df_min['significant'] == True) &
        (abs(mean_df_min['log2_fold_change']) <= log_threshold) &
        (mean_df_min['euclidean_distance'] >= percentile_95)
    ]

    # Highlight the top ten genes by Euclidean distance
    top_genes = filtered_genes.nlargest(10, 'euclidean_distance')

    # Prepare figure
    fig = go.Figure()

    # Add scatter plot for filtered genes
    fig.add_trace(go.Scatter(
        x=filtered_genes['log2_fold_change'],
        y=filtered_genes['euclidean_distance'],
        mode='markers',
        name='Significant Genes',
        hovertext=filtered_genes.apply(lambda row: f"{row['gene_name']} ({row['comparison']})", axis=1),
        marker=dict(color='grey')
    ))

    # Highlight the top ten genes by Euclidean distance
    fig.add_trace(go.Scatter(
        x=top_genes['log2_fold_change'],
        y=top_genes['euclidean_distance'],
        mode='markers+text',
        name='Top 10 Genes by Euclidean Distance',
        hovertext=top_genes.apply(lambda row: f"{row['gene_name']} ({row['comparison']})", axis=1),
        marker=dict(color='red', size=10),
        text=top_genes['gene_name'],
        textposition="top center"
    ))

    fig.update_layout(
        title='Euclidean Distance vs Log2 FC (Filtered and Highlighted)',
        xaxis_title='Differential Gene Expression (log2 FC)',
        yaxis_title='Per-gene Transcript Abundance Variance Score',
        showlegend=True
    )

    # Show figure
    fig.show()

    # Post the figure online if requested
    if post_online:
        py.plot(fig, filename='Euclidean_Distance_vs_Log2_FC', auto_open=True)

    # Sort and reset index
    mean_df_min = mean_df_min.sort_values(by='euclidean_distance', ascending=False)
    mean_df_min.reset_index(drop=True, inplace=True)

    return mean_df_min


In [66]:
import plotly.graph_objects as go
import pandas as pd

def plot_mean_euclidean_and_logfc(mean_df_min, post_online=False):
    """
    Create a bar graph showing the mean Euclidean distance scores and mean absolute logFC for each comparison group.
    Two bars for each comparison group:
    - Mean Euclidean Distance across all genes
    - Mean Absolute logFC across all genes

    Parameters:
    - mean_df_min: DataFrame containing aggregated gene-level results with Euclidean distances and logFC.
    - post_online: Boolean flag to post the figure online through Plotly Chart Studio.

    Returns:
    - None: Displays a bar graph.
    """
    # Define comparison groups
    comparisons = ['B cell_vs_CLL', 'B cell_vs_CLL_RS', 'B cell_vs_RS', 'CLL_vs_CLL_RS', 'CLL_vs_RS', 'CLL_RS_vs_RS']

    # Data storage for bar graph
    euclidean_means = []
    logfc_means = []

    for comp in comparisons:
        # Filter DataFrame for the specific comparison group
        df_comp = mean_df_min[mean_df_min['comparison'] == comp]

        # Calculate mean Euclidean distance across all genes
        mean_euclidean = df_comp['euclidean_distance'].mean()
        euclidean_means.append(mean_euclidean)

        # Calculate mean absolute logFC across all genes
        mean_abs_logfc = df_comp['log2_fold_change'].abs().mean()
        logfc_means.append(mean_abs_logfc)

    # Create the bar graph
    fig = go.Figure(data=[
        go.Bar(name='Mean Euclidean Distance (All Genes)', x=comparisons, y=euclidean_means),
        go.Bar(name='Mean Absolute logFC (All Genes)', x=comparisons, y=logfc_means)
    ])

    # Customize the layout
    fig.update_layout(
        title='Mean Euclidean Distances and Mean Absolute logFC Across Comparison Groups',
        xaxis_title='Comparison Group',
        yaxis_title='Mean Value',
        barmode='group'
    )

    # Show the figure
    fig.show()

    # Post the figure online if requested
    if post_online:
        py.plot(fig, filename='Mean_Euclidean_Distances_and_logFC', auto_open=True)


## Specific Implementation Code

### setting up dictionaries

In [24]:
# # import in GSE17025.top.table.tsv from data
# top_table1 = pd.read_csv('../../data/GSE17025.top.table.tsv', sep='\t')
# # import in GSE63678.top.table.tsv from data
# top_table2 = pd.read_csv('../../data/GSE63678.top.table.tsv', sep='\t')
# # import in GSE115810.top.table.tsv from data
# top_table3 = pd.read_csv('../../data/GSE115810.top.table.tsv', sep='\t')
# # preview the dataframe
# top_table1.head()
# # stack the dataframes on top of each other
# top_table_x = pd.concat([top_table1, top_table2, top_table3])
# # length of the dataframe
# len(top_table_x)

In [25]:
# # isolate the Gene.symbol column
# top_table = top_table_x['Gene.symbol']
# # preview the dataframe
# top_table.head()

In [26]:
# # only keep Gene.symbol and Gene.title columns
# top_table_y = top_table_x[['Gene.symbol', 'Gene.title']]
# # turn the table into a dictionary
# top_table_dict = top_table_y.set_index('Gene.symbol')['Gene.title'].to_dict()

In [27]:
# # find common entries between gene_to_transcript_mapping keys and top_table
# common_genes = set(gene_to_transcript_mapping.keys()).intersection(set(top_table))
# # print the number of common genes
# print('Number of common genes: ', len(common_genes))

In [28]:
# # only keep common genes in gene_to_transcript_mapping
# gene_to_transcript_mapping_common = {gene: gene_to_transcript_mapping[gene] for gene in common_genes}
# # print the length of gene_to_transcript_mapping_common
# print('Length of gene_to_transcript_mapping_common: ', len(gene_to_transcript_mapping_common))

### running the pipeline

In [29]:
# rerun the modded pipeline with the common genes
TPM_dataframe1, TPM_dataframe2, TPM_dataframe3, TPM_dataframe4 = modded_query_maker(metadata, TPM)

Index(['read_group_id', 'ENSMUST00000103569', 'ENSMUST00000103570',
       'ENSMUST00000103571', 'ENSMUST00000198019', 'ENSMUST00000179789',
       'ENSMUST00000178768', 'ENSMUST00000103580', 'ENSMUST00000103646',
       'ENSMUST00000197754'],
      dtype='object')
Dataframe 1 shape:  (11, 99469)
Dataframe 2 shape:  (4, 99469)
Dataframe 3 shape:  (5, 99469)
Dataframe 4 shape:  (4, 99469)


In [68]:
# run comprehensive_gene_analysis
mean_df_min = comprehensive_gene_analysis(transcript_to_gene_mapping, TPM_dataframe1, TPM_dataframe2, TPM_dataframe3, TPM_dataframe4, gene_to_transcript_mapping, 2, 0.1)

Calculating Log FC and Euclidean Distances: 100%|██████████| 35330/35330 [01:27<00:00, 403.10gene/s]


In [69]:
# Example usage
plot_mean_euclidean_and_logfc(mean_df_min)